# Imports and load data

In [1]:
import os
import pandas as pd
import numpy as np
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics      import mean_absolute_error

In [2]:
# GLOBAL VARIABLES
DATA_PATH = '../data'
TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
VALID_PATH = os.path.join(DATA_PATH, 'valid.csv')

CATEGORY_COLUMNS = ['credit_card_level', 'aff_type', 'country_segment']
DATE_COLUMNS = ['join_date']
NUMBER_COLUMNS = ['hidden', 'STV', 'target', 'is_lp', 'is_cancelled']

NULL_CATEGORY_KEY = 'Unknown'
NULL_CATEGORY_VALUE = -1

NULL_NUMBER_VALUE = -1
NULL_DATE_VALUE = datetime.date(1000,1,1)

DTYPE_CATEGORY_COLUMNS = {'credit_card_level': 'category', 
                          'aff_type': 'category', 
                          'country_segment': 'category'}

In [3]:
def read_csv(path):
    return pd.read_csv(path, 
                    parse_dates = DATE_COLUMNS, 
                    dtype = DTYPE_CATEGORY_COLUMNS)
# Read Data
train = read_csv(TRAIN_PATH)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371727 entries, 0 to 371726
Data columns (total 10 columns):
user_id              371727 non-null int64
join_date            371727 non-null datetime64[ns]
hidden               371727 non-null int64
STV                  371727 non-null float64
target               371727 non-null float64
credit_card_level    371727 non-null category
is_lp                371727 non-null int64
aff_type             371727 non-null category
is_cancelled         371727 non-null int64
country_segment      371727 non-null category
dtypes: category(3), datetime64[ns](1), float64(2), int64(4)
memory usage: 20.9 MB


# Preprocessing

In [5]:
def create_or_update_map_from_categories(data, columns, map_categories = None):
    global NULL_CATEGORY_KEY, NULL_CATEGORY_VALUE
    
    if map_categories is None:
        map_categories = dict()
        for col in columns:
            labels = data[col].astype('category').cat.categories.tolist()
            map_categories[col] = {k: v for k,v in zip(labels, list(range(1,len(labels)+1)))}
            map_categories[col][NULL_CATEGORY_KEY] = NULL_CATEGORY_VALUE
    else : 
        for col in columns:
            labels = data[col].astype('category').cat.categories.tolist()
            new_labels = [label for label in labels if label not in list(map_categories[col].keys())]
            if new_labels:
                next_value = max(map_categories[col].values())+1
                map_categories[col].update({k: v for k,v in zip(new_labels,
                                                                list(range(next_value, next_value+len(new_labels)+1)))})
    return map_categories


def preprocessing(data, map_categories = None):
    global CATEGORY_COLUMNS, DATE_COLUMNS, NUMBER_COLUMNS, NULL_DATE_VALUE, NULL_NUMBER_VALUE
    preprocessed = data.copy()
    
    for date_column in DATE_COLUMNS:
        # Check NaT values and convert them to timestamp
        preprocessed[date_column] = preprocessed[date_column].apply(
                                    lambda x: NULL_DATE_VALUE.timestamp() 
                                                if x is pd.NaT else x.timestamp())
    
    for category_column in CATEGORY_COLUMNS:
        # Check NaN values and convert them to NULL_CATEGORY_KEY
        preprocessed[category_column] = preprocessed[category_column].cat \
                                                .add_categories(NULL_CATEGORY_KEY) \
                                                .fillna(NULL_CATEGORY_KEY)

        
    # Convert categories to codes
    map_categories = create_or_update_map_from_categories(data, CATEGORY_COLUMNS, map_categories)
    preprocessed.replace(map_categories, inplace = True)
    
    for number_column in NUMBER_COLUMNS:
         # Check NaN values and convert them to NULL_NUMBER_VALUE
        preprocessed[number_column] = preprocessed[number_column].apply(
                                            lambda x: NULL_NUMBER_VALUE if pd.isna(x) else x)
    
    
    return preprocessed, map_categories

In [6]:
train, map_categories = preprocessing(train)

In [7]:
map_categories

{'credit_card_level': {'prepaid': 1, 'standard': 2, 'Unknown': -1},
 'aff_type': {'Other': 1, 'PPL': 2, 'PPS': 3, 'Unknown': -1},
 'country_segment': {'AU': 1,
  'CL': 2,
  'EC': 3,
  'GB': 4,
  'ID': 5,
  'IL': 6,
  'IN': 7,
  'KZ': 8,
  'MY': 9,
  'NG': 10,
  'NZ': 11,
  'RU': 12,
  'SA': 13,
  'SG': 14,
  'TH': 15,
  'TR': 16,
  'UA': 17,
  'US': 18,
  'ZA': 19,
  'Unknown': -1}}

In [8]:
train.head()

,user_id,join_date,hidden,STV,target,credit_card_level,is_lp,aff_type,is_cancelled,country_segment
0,0,1.543622e+09,1,0.3975,0.3975,1,0,3,1,18
1,1,1.543622e+09,0,0.3975,0.3975,2,0,3,1,18
2,2,1.543622e+09,0,0.3975,0.3975,2,0,3,1,18
3,11,1.543623e+09,1,0.3975,0.3975,1,0,2,1,18
4,17,1.543623e+09,0,0.3975,0.3975,2,0,2,0,18


# Features selection

### Functions definitions

In [9]:
def calculateVIF(data) :
    """
    Calculate VIF (Variance Inflation Factor) of data. 
    
    VIF allows us cuantify the intensity of multicollinearity. The VIF value increases according
    to the multicollinearity increases. The VIF values bigger than 5 are considered high and
    VIF values bigger than 10 are considered very high.
    
    Params:
    data -- data.
    
    Returns:
    A dataframe with the VIF value of each feature.
    """
    features = list(data.columns)
    num_features = len(features)
    
    # Create the model and the result dataframe
    model = LinearRegression()
    result = pd.DataFrame(index = ['VIF'], columns = features)
    result = result.fillna(0)
    
    # For each feature
    for ite in range(num_features) :
        x_features = features[:]
        y_feature  = features[ite]
        # Remove the feature (because it is the independient)
        x_features.remove(y_feature)
        
        x = data[x_features]
        y = data[y_feature]
        
        # Fit the model 
        model.fit(x, y)
        # Calculate VIF
        result[y_feature] = 1 / (1 - model.score(x, y))
    
    return result


def selectFeaturesUsingVIF(data, max_VIF = 5) :
    """
    Select features using its VIF value.
    
    Params:
    data -- data.
    max_VIF -- maximum VIF value to follow removing features.
    
    Returns:
    A dataframe with the features selected.
    """
    
    # Copy data
    result = data.copy(deep = True)
    
    # Calculate VIF of all features
    VIF = calculateVIF(result)
    
    # While the VIF value is bigger than max_VIF:
    while VIF.values.max() > max_VIF :
        # Get the column of the feature which gets the maximum VIF
        col_max = np.where(VIF == VIF.values.max())[1][0]
        
        # Remove this feature of the data
        features = list(result.columns)
        features.remove(features[col_max])
        result = result[features]
        
        # Again, calculate VIF
        VIF = calculateVIF(result)

    # Return the result
    return result

In [10]:
target_name = 'target'
features_list = list(train.columns)
features_list.remove(target_name)

train_target = train[target_name]
train_features = train[features_list]

In [11]:
train_features = selectFeaturesUsingVIF(train_features)
train_features.head()

,join_date,hidden,STV,credit_card_level,is_lp,aff_type,is_cancelled,country_segment
0,1.543622e+09,1,0.3975,1,0,3,1,18
1,1.543622e+09,0,0.3975,2,0,3,1,18
2,1.543622e+09,0,0.3975,2,0,3,1,18
3,1.543623e+09,1,0.3975,1,0,2,1,18
4,1.543623e+09,0,0.3975,2,0,2,0,18


# Model training

In [12]:
# Create a model and fit it with de train data
model = LinearRegression()
model.fit(train_features, train_target)

# Let's predict the train data
pred = model.predict(train_features)

print("The model's metrics are:")
print('Trainning R^2 = ', model.score(train_features, train_target))
print('MAE = ', mean_absolute_error(pred, train_target))

The model's metrics are:
Trainning R^2 =  0.6376326892097208
MAE =  3.717517283540668


# Model validation (all dataset)

In [13]:
# Read valid dataset
valid = read_csv(VALID_PATH)
# Preprocessing
valid, map_categories = preprocessing(valid, map_categories)
map_categories

{'credit_card_level': {'prepaid': 1, 'standard': 2, 'Unknown': -1},
 'aff_type': {'Other': 1, 'PPL': 2, 'PPS': 3, 'Unknown': -1},
 'country_segment': {'AU': 1,
  'CL': 2,
  'EC': 3,
  'GB': 4,
  'ID': 5,
  'IL': 6,
  'IN': 7,
  'KZ': 8,
  'MY': 9,
  'NG': 10,
  'NZ': 11,
  'RU': 12,
  'SA': 13,
  'SG': 14,
  'TH': 15,
  'TR': 16,
  'UA': 17,
  'US': 18,
  'ZA': 19,
  'Unknown': -1,
  'CA': 20}}

In [14]:
valid_features = valid[train_features.columns]
valid_target = valid[target_name]

# Let's predict the valid data
pred = model.predict(valid_features)

print("The model's metrics are:")
print('Validation R^2 = ', model.score(valid_features, valid_target))
print('MAE = ', mean_absolute_error(pred, valid_target))

The model's metrics are:
Validation R^2 =  0.6274331639546528
MAE =  4.1402298970781795


# Model validation (aggregated dataset)

In [15]:
# Read valid dataset
valid = read_csv(VALID_PATH)

# Truncate join_date column by day/month/year
valid['join_date'] = valid['join_date'].apply(lambda x: x.replace(hour=0, minute=0, second=0, microsecond=0))

# Aggregate dataset
valid_aggregated = valid.groupby(['join_date', 'country_segment', 'hidden', 
                          'credit_card_level', 'is_lp', 
                          'aff_type', 'is_cancelled'], as_index = False).agg({'STV': 'sum', 'target': 'sum'})

valid_aggregated.head()

,join_date,country_segment,hidden,credit_card_level,is_lp,aff_type,is_cancelled,STV,target
0,2019-07-01,AU,0,standard,0,PPL,0.0,21.445125,126.265875
1,2019-07-01,AU,0,standard,0,PPL,1.0,0.795000,0.795000
2,2019-07-01,AU,0,standard,0,PPS,0.0,144.928500,312.269750
3,2019-07-01,AU,0,standard,0,PPS,1.0,20.252625,20.252625
4,2019-07-01,AU,0,standard,1,PPL,0.0,1.590000,2.385000


In [16]:
# Preprocessing
valid_aggregated, map_categories = preprocessing(valid_aggregated, map_categories)
valid_aggregated.head()

,join_date,country_segment,hidden,credit_card_level,is_lp,aff_type,is_cancelled,STV,target
0,1.561939e+09,1,0,2,0,2,0.0,21.445125,126.265875
1,1.561939e+09,1,0,2,0,2,1.0,0.795000,0.795000
2,1.561939e+09,1,0,2,0,3,0.0,144.928500,312.269750
3,1.561939e+09,1,0,2,0,3,1.0,20.252625,20.252625
4,1.561939e+09,1,0,2,1,2,0.0,1.590000,2.385000


In [17]:
country_segment = 'country_segment'
country_codes = map_categories[country_segment]

for country in country_codes:
    valid_features = valid_aggregated[valid_aggregated[country_segment] == country_codes[country]][train_features.columns]
    valid_target = valid_aggregated[valid_aggregated[country_segment] == country_codes[country]][target_name]
    
    if not valid_features.empty:
        # Let's predict the valid data
        pred = model.predict(valid_features)

        print("The model's metrics for country", country, "are:")
        print('Validation R^2 = ', model.score(valid_features, valid_target))
        print('MAE = ', mean_absolute_error(pred, valid_target))
        print()

The model's metrics for country AU are:
Validation R^2 =  0.7458420968801912
MAE =  24.125359883300575

The model's metrics for country CL are:
Validation R^2 =  0.7215845005703281
MAE =  7.081910658629403

The model's metrics for country EC are:
Validation R^2 =  0.7534477949728466
MAE =  5.425747113271214

The model's metrics for country GB are:
Validation R^2 =  0.6703831326009788
MAE =  15.80245919449084

The model's metrics for country ID are:
Validation R^2 =  0.7803875980890411
MAE =  7.399590443862857

The model's metrics for country IL are:
Validation R^2 =  0.6586716338606055
MAE =  34.829318285528515

The model's metrics for country IN are:
Validation R^2 =  0.7792546970004206
MAE =  3.9791495464559072

The model's metrics for country KZ are:
Validation R^2 =  0.7631446208488026
MAE =  5.231649132772522

The model's metrics for country MY are:
Validation R^2 =  0.7338599823245908
MAE =  20.64396917259928

The model's metrics for country NG are:
Validation R^2 =  0.7156602891

# Model saving

In [18]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))